# Model saving/loading/plotting
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

```python
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
```

In [1]:
from keras.models import model_from_json
json_file = open('NLPmodel.json')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json )

loaded_model.load_weights('NLPmodel.h5')

loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
loaded_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2169, 300)         112200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2010      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 515,021
Trainable params: 402,821
Non-trainable params: 112,200
_________

In [11]:
loaded_model.weights

[<tf.Variable 'lstm_1/kernel:0' shape=(300, 800) dtype=float32_ref>,
 <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(200, 800) dtype=float32_ref>,
 <tf.Variable 'lstm_1/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(200, 10) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(10, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'embedding_1/embeddings:0' shape=(374, 300) dtype=float32_ref>]

In [10]:
from keras.utils import plot_model
plot_model(loaded_model,show_shapes=True, rankdir='TB',to_file='nlp_model.png')

In [7]:
# from IPython.display import SVG
# from keras.utils import model_to_dot

# SVG(model_to_dot(loaded_model).create(prog='dot', format='svg'))

In [1]:
## NEW EVAL METHOD USING PREDICTIONS AS PART OF SLIDING WINDOW
    # # GETTING THE FIRST EVAL PATCH AND RESHAPING TO MATCH INPUT
    # first_eval_batch = train_data[-n_input:]
    # current_batch = first_eval_batch.reshape((1, n_input, n_features))
    # test_predictions=[]
    # # LOOP THROUGH ALL TEST DATA
    # for i in range(len(test_data)):

    #     # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    #     current_pred = model.predict(current_batch)[0] 

    #     # store prediction
    #     test_predictions.append(current_pred) 

    #     # UPDATE BATCH TO INCLUDE CURRENT PREDICTION AND DROP OLDEST VALUE FROM ARRAY
    #     current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1) # axis===ndim

CODE THAT IS BEING REPLACE BY TIMESERIESGENERATOR
```python
# BIN STOCK_DF BY INTO X_WINDOW # TIME SEQUENCES
# Create binned columns with x_window # of sequential values, label column with next value in sequence
stock_df_binned = make_df_timeseries_bins_by_column(stock_df, x_window=x_window, verbose=0)#.iloc[:,11:])
# display(stock_df_binned.head(2).style.set_caption('Binned for Neural Network'))


# Split the training and test data by number of days
df_train_bins, df_test_bins = train_test_split_by_last_days(stock_df_binned,periods_per_day=periods_per_day,
                                                            num_test_days=num_test_days,num_train_days=num_train_days,
                                                            plot=True)
```


- And then no need to create X_train, y_train, etc

```python
# Creating X,y and index for training data and test data
train_index =df_train_bins.index
test_index = df_test_bins.index

X_train = df_train_bins['price_bins'].values
y_train = df_train_bins['price_labels'].values

X_test = df_test_bins['price_bins'].values
y_test = df_test_bins['price_labels'].values


# Converting all X and y into 2D arrays
X_train_stack =  np.vstack(X_train)
X_train_in = np.reshape(X_train_stack,(X_train_stack.shape[0],X_train_stack.shape[1],1))

X_test_stack =  np.vstack(X_test)
X_test_in = np.reshape(X_test_stack,(X_test_stack.shape[0],X_test_stack.shape[1],1))


# Set True to display array details
show_array_details = False

# Display details of array data, if desired
if show_array_details==True:
    var_dict = {'X_train':X_train_in, 'y_train':y_train,'X_test':X_test_in,'y_test':y_test}
    for name, arr in var_dict.items():
        print_array_inf(arr,name)
```

- Then model.fit replaced with fit_generator

```python
history = model.fit(X_train_in, y_train, epochs=3, verbose=2, validation_split=(0.1))
```

#### USING ROLLBACK - PANDAS GUIDE
- from the https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

```python
In [148]: ts = pd.Timestamp('2018-01-06 00:00:00')

In [149]: ts.day_name()
Out[149]: 'Saturday'

# BusinessHour's valid offset dates are Monday through Friday
In [150]: offset = pd.offsets.BusinessHour(start='09:00')

# Bring the date to the closest offset date (Monday)
In [151]: offset.rollforward(ts)
Out[151]: Timestamp('2018-01-08 09:00:00')

# Date is brought to the closest offset date first and then the hour is added
In [152]: ts + offset
Out[152]: Timestamp('2018-01-08 10:00:00')
```